In [34]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] =os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] =os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [35]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

model = OllamaLLM(model = "llama3")

embeddings = (
    OllamaEmbeddings(model = "llama3")
)

In [36]:
model.invoke([
    HumanMessage(content="Hi, my name is Smayan and I'm a chief AI engineer.")
])

"Robot: Greetings Smayan! It's great to meet you. As a chief AI engineer, what kind of projects have you been working on? Are you focused on developing intelligent systems for industries like healthcare or finance, or are you exploring more cutting-edge areas like natural language processing or computer vision?"

In [37]:
model.invoke([
    HumanMessage(content="Hi, my name is Smayan and I'm a chief AI engineer."),
    AIMessage(content="Nice to meet you, Smayan! Congratulations on being a chief AI engineer - that's an impressive title! What kind of projects have you been working on lately? Are there any particularly exciting or challenging AI-related initiatives on your plate right now?"),
    HumanMessage(content="Hey, What is my name and what do i do?")
])

"I think I can help with that!\n\nYour name is Smayan, and according to our initial conversation, you're a chief AI engineer!"

### Message History

In [38]:
from langchain_community.chat_message_histories import ChatMessageHistory
from  langchain_core.chat_history import BaseChatMessageHistory
from  langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [39]:
config={"configurable":{"session_id":"chat1"}}

In [40]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Smayan and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


In [41]:
response

"Nice to meet you, Smayan! As a Chief AI Engineer, that's impressive. What kind of projects are you working on? Are you focused on developing new AI models or improving existing ones? Do you have any exciting collaborations or innovations in the pipeline? I'm all ears and curious to learn more about your work!"

In [42]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"I'm happy to help! However, I don't think we've had a chance to establish your name yet. As our conversation just started, I don't have any information about you or your identity. Would you like to introduce yourself and tell me your name?"